Create a web scraper which scraps information from the web and stores it in a database. Specifically develop a model which scrap information regarding top 250 movies from IMDB and stores it in a mysql database. Use it to generate useful analytics.

In [3]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [4]:
url = 'http://www.imdb.com/chart/top'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [5]:
movies = soup.select('td.titleColumn')
crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]
ratings = [b.attrs.get('data-value')
        for b in soup.select('td.posterColumn span[name=ir]')]

In [6]:
list = []

for index in range(0, len(movies)):
    movie_string = movies[index].get_text()
    movie = (' '.join(movie_string.split()).replace('.', ''))
    movie_title = movie[len(str(index))+1:-7]
    year = re.search('\((.*?)\)', movie_string).group(1)
    place = movie[:len(str(index))-(len(movie))]
    data = {"place": place,
            "movie_title": movie_title,
            "rating": ratings[index],
            "year": year,
            "star_cast": crew[index],
            }
    list.append(data)

In [7]:
"""
for movie in list:
    print(movie['place'], '-', movie['movie_title'], '('+movie['year'] +
          ') -', 'Starring:', movie['star_cast'], movie['rating'])
"""

"\nfor movie in list:\n    print(movie['place'], '-', movie['movie_title'], '('+movie['year'] +\n          ') -', 'Starring:', movie['star_cast'], movie['rating'])\n"

In [8]:
df = pd.DataFrame(list)
df.to_csv('imdb_top_250_movies.csv',index=False)
df

,place,movie_title,rating,year,star_cast
0,1,The Shawshank Redemption,9.235860540453793,1994,"Frank Darabont (dir.), Tim Robbins, Morgan Fre..."
1,2,The Godfather,9.155771307298233,1972,"Francis Ford Coppola (dir.), Marlon Brando, Al..."
2,3,The Dark Knight,8.991993373502519,2008,"Christopher Nolan (dir.), Christian Bale, Heat..."
3,4,The Godfather Part II,8.983827579082813,1974,"Francis Ford Coppola (dir.), Al Pacino, Robert..."
4,5,12 Angry Men,8.953503188723783,1957,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb"
...,...,...,...,...,...
245,246,The Iron Giant,8.006130201534289,1999,"Brad Bird (dir.), Eli Marienthal, Harry Connic..."
246,247,The Help,8.005687958303934,2011,"Tate Taylor (dir.), Viola Davis, Emma Stone"
247,248,Aladdin,8.003740059591323,1992,"Ron Clements (dir.), Scott Weinger, Robin Will..."
248,249,Dersu Uzala,8.003204407118897,1975,"Akira Kurosawa (dir.), Maksim Munzuk, Yuriy So..."


In [9]:
import mysql.connector
import pyodbc
conn = mysql.connector.connect(host='localhost',
                                       database='dbmslab',
                                       user='root',
                                       password='Geeky@Zain98')
cursor=conn.cursor(buffered=True)

In [10]:
print(df.head(5))

  place               movie_title             rating  year  \
0     1  The Shawshank Redemption  9.235860540453793  1994   
1     2             The Godfather  9.155771307298233  1972   
2     3           The Dark Knight  8.991993373502519  2008   
3     4     The Godfather Part II  8.983827579082813  1974   
4     5              12 Angry Men  8.953503188723783  1957   

                                           star_cast  
0  Frank Darabont (dir.), Tim Robbins, Morgan Fre...  
1  Francis Ford Coppola (dir.), Marlon Brando, Al...  
2  Christopher Nolan (dir.), Christian Bale, Heat...  
3  Francis Ford Coppola (dir.), Al Pacino, Robert...  
4      Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb  


In [26]:
cursor.execute('''create table webscraping
                (place int,movie_title text,rating double DEFAULT NULL,year int,star_cast text)''')


In [27]:
for row in df.itertuples():
    query="INSERT INTO webscraping (place,movie_title,rating,year,star_cast) VALUES (%s,%s,%s,%s,%s)"
    cursor.execute(query,(row.place,row.movie_title, row.rating,row.year,row.star_cast))


In [32]:
cursor.execute('select * from webscraping')
records=cursor.fetchall()
for x in records:
    print(x)

(1, 'The Shawshank Redemption', 9.235857621446582, 1994, 'Frank Darabont (dir.), Tim Robbins, Morgan Freeman')
(2, 'The Godfather', 9.155772967990414, 1972, 'Francis Ford Coppola (dir.), Marlon Brando, Al Pacino')
(3, 'The Dark Knight', 8.991957602034253, 2008, 'Christopher Nolan (dir.), Christian Bale, Heath Ledger')
(4, 'The Godfather Part II', 8.983807982931284, 1974, 'Francis Ford Coppola (dir.), Al Pacino, Robert De Niro')
(5, '12 Angry Men', 8.953460266380482, 1957, 'Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb')
(6, "Schindler's List", 8.940410970601459, 1993, 'Steven Spielberg (dir.), Liam Neeson, Ralph Fiennes')
(7, 'The Lord of the Rings: The Return of the King', 8.932883500017175, 2003, 'Peter Jackson (dir.), Elijah Wood, Viggo Mortensen')
(8, 'Pulp Fiction', 8.846012842871843, 1994, 'Quentin Tarantino (dir.), John Travolta, Uma Thurman')
(9, 'The Lord of the Rings: The Fellowship of the Ring', 8.811331213951693, 2001, 'Peter Jackson (dir.), Elijah Wood, Ian McKellen')
(1, 

In [33]:
cursor.execute('commit')